In [1]:
1+1

2

In [2]:
import mozo2 

import importlib
importlib.reload(mozo2)

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from mozo import compare_and_combine_2, Diff, merge_with_features, num_to_range_categories
import pandas as pd
import numpy as np
np.random.seed(0)

import xgboost as xgb

from scipy import stats
import math 
import datetime 

from sklearn.metrics import r2_score, mean_absolute_error
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from functools import partial
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from sklearn.svm import SVC

from sklearn.metrics import r2_score, f1_score, mean_absolute_error, mean_squared_error

from scikitplot.estimators import plot_learning_curve

import time
from datetime import datetime


In [3]:
total = pd.read_csv('~/pml7/konkurs/output/newest_total_20201101_07_20.h5.csv')
total.info(verbose=True)

/home/conda/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,86,161) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160616 entries, 0 to 160615
Data columns (total 218 columns):
 #   Column                                                  Dtype  
---  ------                                                  -----  
 0   breadcrumb                                              object 
 1   created_at                                              object 
 2   price_currency                                          object 
 3   price_details                                           object 
 4   price_value                                             float64
 5   seller_address                                          object 
 6   seller_name                                             object 
 7   seller_type                                             object 
 8   feature_czujniki_parkowania_przednie                    bool   
 9   feature_poduszka_powietrzna_chroniąca_kolana            bool   
 10  feature_kurtyny_powietrzne                             

print_plots_by_type(total, type =[np.object])

In [4]:
## modelowanie

In [5]:
train_30 = pd.read_hdf('output/train_30.h5', 'data') 
test_car_ids = train_30['car_id'].values

train_70 = total[~total['car_id'].isin(test_car_ids) & ~total['price_value'].isna()]
train_30 = total[total['car_id'].isin(test_car_ids)]

print(len(total.index))
print(len(total[~total['price_value'].isna()].index))

print(train_70['price_value'].isna().sum())
print(train_30['price_value'].isna().sum())

train_100 = total[~total['price_value'].isna()]
print(train_100['price_value'].isna().sum())

global_min = total[~total['price_value'].isna()].price_value.min()

test = total[total['price_value'].isna()]
train = total[~total['price_value'].isna()]


160616
106494
0
0
0


In [6]:
start_vars = np.sort([

# 'created_at_unix', 'encoded_param_rodzaj_paliwa_2', 'encoded_param_typ_2', 'feature_abs', 'feature_asr__kontrola_trakcji_', 'feature_bluetooth', 'feature_cd', 'feature_czujniki_parkowania_przednie', 'feature_elektrycznie_ustawiane_fotele', 'feature_elektrycznie_ustawiane_lusterka', 'feature_gniazdo_sd', 'feature_hud__wyświetlacz_przezierny_', 'feature_isofix', 'feature_klimatyzacja_automatyczna', 'feature_klimatyzacja_manualna', 'feature_ogrzewanie_postojowe', 'feature_podgrzewane_tylne_siedzenia', 'feature_relingi_dachowe', 'feature_tapicerka_skórzana', 'feature_tapicerka_welurowa', 'feature_tempomat', 'feature_światła_led', 'feature_światła_przeciwmgielne', 'feature_światła_xenonowe', 'latitude_na_mean', 'longitude', 'longitude_na_mean', 'model_8790', 'new_param_kolor_cat', 'new_param_liczba_pozostałych_rat_cat', 'new_param_marka_pojazdu_cat', 'new_param_oferta_od_cat', 'new_param_opłata_początkowa_cat', 'new_param_rodzaj_paliwa_cat', 'new_param_wersja_cat', 'new_seller_name_cat', 'new_vin_flex_fuel_cat', 'new_vin_model_cat', 'new_vin_security_cat', 'param_bezwypadkowy', 'param_homologacja_ciężarowa', 'param_ile_lat_temu_produkcja', 'param_leasing', 'param_liczba_miejsc_2', 'param_liczba_pozostałych_rat_2', 'param_matowy', 'param_przebieg', 'param_rok_produkcji', 'param_tuning', 'param_uszkodzony', 'param_wartość_wykupu_2'
'model_8790', 'model_XGBRegressor_2608_37590_po_hiperopcie', 'created_at_unix', 'encoded_param_rodzaj_paliwa_2', 'encoded_param_skrzynia_biegów_2', 'encoded_param_typ_2', 'feature_asr__kontrola_trakcji_', 'feature_cd', 'feature_centralny_zamek', 'feature_czujnik_martwego_pola', 'feature_elektrycznie_ustawiane_fotele', 'feature_elektrycznie_ustawiane_lusterka', 'feature_gniazdo_sd', 'feature_isofix', 'feature_klimatyzacja_automatyczna', 'feature_klimatyzacja_manualna', 'feature_ogrzewanie_postojowe', 'feature_podgrzewane_tylne_siedzenia', 'feature_relingi_dachowe', 'feature_tapicerka_skórzana', 'feature_światła_led', 'feature_światła_przeciwmgielne', 'longitude', 'longitude_na_mean', 'new_param_kolor_cat', 'new_param_liczba_pozostałych_rat_cat', 'new_param_marka_pojazdu_cat', 'new_param_oferta_od_cat', 'new_param_opłata_początkowa_cat', 'new_seller_name_cat', 'new_vin_flex_fuel_cat', 'new_vin_model_cat', 'new_vin_portrait_cat', 'new_vin_security_cat', 'ogloszenie_rok', 'param_akryl__niemetalizowany_', 'param_bezwypadkowy', 'param_homologacja_ciężarowa', 'param_ile_lat_temu_produkcja', 'param_liczba_miejsc_2', 'param_liczba_pozostałych_rat_2', 'param_matowy', 'param_moc', 'param_perłowy', 'param_pierwsza_rejestracja_10_lat', 'param_pojemność_skokowa', 'param_przebieg', 'param_rok_produkcji', 'param_tuning', 'param_uszkodzony', 'param_vin_len'
])

# 'model_8790' = car_price_log_Vladimir_categories_xgb.csv; 5637.56722; "opis": "Vladimir z kategorialnymi mae na train_30"

black_list = ['car_id', 'price_details', 'price_value', 'price_value_log', 'price_value_pln', 'price_value_pln_log_transl', 'new_cat_price_value',
#               *[x for x in total.columns if x.startswith('model_')],
              'model_XGBRegressor_2195_24982_po_hiperopcie_caly_train', #train_100, przeuczony
              'model_XGBRegressor_3293_22097_', #train_100
              'longitude_na_mean', 'latitude_na_mean', #gorsze w xbfir
              *total.columns[total.isnull().any()],
             ]

feats3 = total.select_dtypes(include=[np.number, np.bool]).columns
feats3 = [x for x in feats3 if x not in black_list]
feats3 = np.sort(feats3)
print(total[feats3].info(verbose=True))
print(total[feats3].columns[total[feats3].isnull().any()])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160616 entries, 0 to 160615
Data columns (total 163 columns):
 #   Column                                              Dtype  
---  ------                                              -----  
 0   created_at_unix                                     float64
 1   created_at_unix_na_mean                             float64
 2   encoded_param_kolor_2                               int64  
 3   encoded_param_napęd_2                               int64  
 4   encoded_param_rodzaj_paliwa_2                       int64  
 5   encoded_param_skrzynia_biegów_2                     int64  
 6   encoded_param_typ_2                                 int64  
 7   encoded_seller_miasto_297                           int64  
 8   feature_abs                                         bool   
 9   feature_alarm                                       bool   
 10  feature_alufelgi                                    bool   
 11  feature_asr__kontrola_trakcji_        

In [ ]:
import contextlib
import warnings
warnings.filterwarnings("ignore")
from catboost import CatBoostRegressor
import json

# params = {                                                                                     
#     "iterations": 704,
#     "depth": 6,
#     "learning_rate": 0.23701660211398967,
#     "random_strength": 0.13419513355474952,
#     "bagging_temperature": 0.29043879590002025,
#     "border_count": 78,
#     "l2_leaf_reg": 10,
#     "random_state": 6950,
#     "silent": True
# }
params = {                                                                                     
    "iterations": 200,
    "depth": 8,
#     "learning_rate": 0.23701660211398967,
#     "random_strength": 0.13419513355474952,
#     "bagging_temperature": 0.29043879590002025,
#     "border_count": 78,
#     "l2_leaf_reg": 10,
    "random_state": 6950,
    "silent": True
}


mae_global_min = 2778.115628209074 
big_iter = 0

current_vars=start_vars
all_vars = feats3 #maksymalny set

while True:

    pairs_so_far = []

    last_added = None
    last_removed = None

    last_trial = False
    big_iter += 1
    print("START iter=" + str(big_iter) + ', mae_global_min=' + str(mae_global_min))
    
    while True:
        with contextlib.redirect_stdout(None):
            current_vars, last_added, last_removed, pairs_so_far, last_trial =  mozo2.mutate_rand_feature_pairs(current_vars, all_vars, pairs_so_far)
#         print('pairs_so_far: ' + json.dumps(pairs_so_far, ensure_ascii=False, indent=4))
        print('added: ' + last_added)
        print('removed: ' + last_removed)
        print('len: ' + str(len(current_vars)))
        
        mae, r2, model, model_cl, learning_curve_plot = mozo2.train_and_submit(train_70, train_30, total.loc[total['price_value'].isna(),], 
                     current_vars, global_min, 
                     CatBoostRegressor, params, digitize=None, subfolder='CatBoostRegressor_parami',
                     file_sufix = 'parami', opis='start=2778.115628209074', 
                     learning_curve = False, 
                     add_model_column_min = -1, total = total, 
                     save_min=mae_global_min, kaggle_min=-1)

        if mae < mae_global_min:
            mae_global_min = mae
            print('Improvement: ' + last_removed + ' -> ' + last_added)
        else:
            current_vars = mozo2.replace_feature(current_vars, last_added, last_removed)
            print('Rollback: ' + last_added + ' -> ' + last_removed)
        print('=============================================' + str(mae_global_min)+ '=============================================')

        if last_trial == True: 
            print("END iter=" + str(big_iter) + ', mae_global_min=' + str(mae))
            break

START iter=1, mae_global_min=2778.115628209074
added: feature_radio_niefabryczne
removed: param_rok_produkcji
len: 51
mea: 3010.3601592841096
r2: 0.9744868797312106
Rollback: feature_radio_niefabryczne -> param_rok_produkcji
=============================================2778.115628209074=============================================
added: new_param_kategoria_cat
removed: encoded_param_typ_2
len: 51
mea: 3013.113041378244
r2: 0.9735018670342981
Rollback: new_param_kategoria_cat -> encoded_param_typ_2
=============================================2778.115628209074=============================================
added: new_param_miesięczna_rata_cat
removed: param_liczba_miejsc_2
len: 51
mea: 3015.2170318538365
r2: 0.9739628992705005
Rollback: new_param_miesięczna_rata_cat -> param_liczba_miejsc_2
=============================================2778.115628209074=============================================
added: feature_bluetooth
removed: feature_światła_przeciwmgielne
len: 51
mea: 3012.358351379

mea: 3000.2402955509165
r2: 0.9747841033274763
Rollback: feature_podgrzewana_przednia_szyba -> created_at_unix
=============================================2778.115628209074=============================================
added: created_at_unix_na_mean
removed: param_bezwypadkowy
len: 51
mea: 3020.7531236603104
r2: 0.9737447544394119
Rollback: created_at_unix_na_mean -> param_bezwypadkowy
=============================================2778.115628209074=============================================
added: feature_wspomaganie_kierownicy
removed: feature_cd
len: 51
mea: 3017.633804466365
r2: 0.9738409195413044
Rollback: feature_wspomaganie_kierownicy -> feature_cd
=============================================2778.115628209074=============================================
added: feature_poduszka_powietrzna_kierowcy
removed: param_liczba_pozostałych_rat_2
len: 51
mea: 2982.946450891688
r2: 0.974783390637171
Rollback: feature_poduszka_powietrzna_kierowcy -> param_liczba_pozostałych_rat_2
==========

mea: 3003.7320045456886
r2: 0.9740765237917476
Rollback: new_param_napęd_cat -> new_vin_flex_fuel_cat
=============================================2778.115628209074=============================================
added: created_at_unix_na_mean
removed: param_uszkodzony
len: 51
mea: 3012.066821240685
r2: 0.9739311554782009
Rollback: created_at_unix_na_mean -> param_uszkodzony
=============================================2778.115628209074=============================================
added: feature_tempomat_aktywny
removed: param_bezwypadkowy
len: 51
mea: 3020.1252363838207
r2: 0.9740394982471824
Rollback: feature_tempomat_aktywny -> param_bezwypadkowy
=============================================2778.115628209074=============================================
added: feature_podgrzewana_przednia_szyba
removed: param_tuning
len: 51
mea: 3010.539584865984
r2: 0.9739995137887438
Rollback: feature_podgrzewana_przednia_szyba -> param_tuning
=============================================2778.11562820

mea: 2987.4395503341666
r2: 0.9746170977364794
Rollback: encoded_seller_miasto_297 -> new_vin_flex_fuel_cat
=============================================2778.115628209074=============================================
added: param_pierwsza_rejestracja_poprzedni_rok
removed: param_perłowy
len: 51
mea: 2994.9195786407017
r2: 0.9742280077706641
Rollback: param_pierwsza_rejestracja_poprzedni_rok -> param_perłowy
=============================================2778.115628209074=============================================
added: feature_czujniki_parkowania_tylne
removed: param_tuning
len: 51
mea: 3015.190448542572
r2: 0.9741950738897646
Rollback: feature_czujniki_parkowania_tylne -> param_tuning
=============================================2778.115628209074=============================================
added: feature_poduszki_boczne_przednie
removed: feature_elektrycznie_ustawiane_fotele
len: 51
mea: 3007.015283346888
r2: 0.9741907920933758
Rollback: feature_poduszki_boczne_przednie -> feature_el

mea: 3008.6870049489953
r2: 0.9740517956787667
Rollback: feature_immobilizer -> encoded_param_typ_2
=============================================2778.115628209074=============================================
added: new_vin_where_cat
removed: param_rok_produkcji
len: 51
mea: 3018.898314104995
r2: 0.9744299234796422
Rollback: new_vin_where_cat -> param_rok_produkcji
=============================================2778.115628209074=============================================
added: encoded_seller_miasto_297
removed: param_moc
len: 51
mea: 3009.6444833238693
r2: 0.973875637961828
Rollback: encoded_seller_miasto_297 -> param_moc
=============================================2778.115628209074=============================================
added: new_param_wartość_wykupu_cat
removed: param_ile_lat_temu_produkcja
len: 51
mea: 3018.326533793128
r2: 0.973898784724226
Rollback: new_param_wartość_wykupu_cat -> param_ile_lat_temu_produkcja
=============================================2778.115628209074==

...

mea: 3007.895650544028
r2: 0.9744472235634094
Rollback: feature_czujniki_parkowania_tylne -> param_ile_lat_temu_produkcja
=============================================2778.115628209074=============================================
added: feature_abs
removed: encoded_param_typ_2
len: 51
mea: 3018.481821022252
r2: 0.9740334131626668
Rollback: feature_abs -> encoded_param_typ_2
=============================================2778.115628209074=============================================
added: feature_esp__stabilizacja_toru_jazdy_
removed: feature_tapicerka_skórzana
len: 51
mea: 3023.282442077783
r2: 0.9739510392011614
Rollback: feature_esp__stabilizacja_toru_jazdy_ -> feature_tapicerka_skórzana
=============================================2778.115628209074=============================================
added: feature_elektrochromatyczne_lusterko_wsteczne
removed: feature_czujnik_martwego_pola
len: 51
mea: 3009.535343103262
r2: 0.9742625800958253
Rollback: feature_elektrochromatyczne_lusterko_w

mea: 3005.4960694835627
r2: 0.9743704272359945
Rollback: feature_hak -> new_vin_model_cat
=============================================2778.115628209074=============================================
added: feature_przyciemniane_szyby
removed: feature_asr__kontrola_trakcji_
len: 51
mea: 3016.425158411015
r2: 0.9737551634065027
Rollback: feature_przyciemniane_szyby -> feature_asr__kontrola_trakcji_
=============================================2778.115628209074=============================================
added: new_cat_seller_address
removed: param_bezwypadkowy
len: 51
mea: 3007.754043184936
r2: 0.9743071649186563
Rollback: new_cat_seller_address -> param_bezwypadkowy
=============================================2778.115628209074=============================================
added: ogloszenie_dni_wisi
removed: param_pierwsza_rejestracja_10_lat
len: 51
mea: 2992.739166998614
r2: 0.9750878348030211
Rollback: ogloszenie_dni_wisi -> param_pierwsza_rejestracja_10_lat
===========================

mea: 3027.5377772121715
r2: 0.9739079762627599
Rollback: feature_poduszki_boczne_tylne -> param_perłowy
=============================================2778.115628209074=============================================
added: feature_hud__wyświetlacz_przezierny_
removed: param_liczba_pozostałych_rat_2
len: 51
mea: 2980.947782621067
r2: 0.9747200393989474
Rollback: feature_hud__wyświetlacz_przezierny_ -> param_liczba_pozostałych_rat_2
=============================================2778.115628209074=============================================
added: feature_wielofunkcyjna_kierownica
removed: model_XGBRegressor_2608_37590_po_hiperopcie
len: 51
mea: 2858.3488035121745
r2: 0.973466043362969
Rollback: feature_wielofunkcyjna_kierownica -> model_XGBRegressor_2608_37590_po_hiperopcie
=============================================2778.115628209074=============================================
added: new_param_skrzynia_biegów_cat
removed: param_matowy
len: 51
mea: 3036.2733518662976
r2: 0.9737647882978439


mea: 3003.8408611953055
r2: 0.9743026697463179
Rollback: ogloszenie_miesiac -> feature_tapicerka_skórzana
=============================================2778.115628209074=============================================
added: feature_asystent_pasa_ruchu
removed: feature_cd
len: 51
mea: 3022.777567255982
r2: 0.9743937805822078
Rollback: feature_asystent_pasa_ruchu -> feature_cd
=============================================2778.115628209074=============================================
added: param_pierwsza_rejestracja_date_na
removed: model_8790
len: 51
mea: 2991.6514865779877
r2: 0.973587980023095
Rollback: param_pierwsza_rejestracja_date_na -> model_8790
=============================================2778.115628209074=============================================
added: new_vin_where_cat
removed: feature_gniazdo_sd
len: 51
mea: 3008.0678335157995
r2: 0.9744464973818873
Rollback: new_vin_where_cat -> feature_gniazdo_sd
=============================================2778.115628209074==============

mea: 3006.9648509942413
r2: 0.9745231262866192
Rollback: param_filtr_cząstek_stałych -> created_at_unix
=============================================2778.115628209074=============================================
added: param_zarejestrowany_w_polsce
removed: encoded_param_typ_2
len: 51
mea: 2998.2256424385196
r2: 0.9743091874440213
Rollback: param_zarejestrowany_w_polsce -> encoded_param_typ_2
=============================================2778.115628209074=============================================
added: feature_wspomaganie_kierownicy
removed: param_homologacja_ciężarowa
len: 51
mea: 3022.233082175624
r2: 0.9738934764524515
Rollback: feature_wspomaganie_kierownicy -> param_homologacja_ciężarowa
=============================================2778.115628209074=============================================
added: feature_asystent_parkowania
removed: feature_relingi_dachowe
len: 51
mea: 3005.474986141569
r2: 0.9741302498059136
Rollback: feature_asystent_parkowania -> feature_relingi_dachowe


In [21]:
model = RandomForestRegressor(n_estimators=50, max_depth=7).fit(X_sub_train, y_sub_train) 
perm = PermutationImportance(model).fit(X_sub_test, y_sub_test) 
eli5.show_weights(perm, feature_names=random_feats, top=50) 


NameError: name 'RandomForestRegressor' is not defined

mae, r2, model, model_cl, learning_curve_plot = train_and_submit(train_70, train_30, total.loc[total['price_value'].isna(),], 
             start_vars, global_min, 
             xgb.XGBRegressor, params, digitize=None, subfolder='',
             file_sufix = 'test', opis='', 
             learning_curve = False, 
             add_model_column_min = -1, total = total, 
             save_min=-1, kaggle_min=-1)


save_model(file_sufix='test', subfolder='', test = total.loc[total['price_value'].isna()],
               feats=current_vars, global_min=global_min, model=model, shift = 0, digitize=False, mae=mae, r2=r2,
               details='test zapisu', learning_curve_plot=None, hist_plt=None, kaggle=None)


today = datetime.datetime.today() 
cvs_file_name = f'newest_total_{today:%Y%m%d_%H_%M.h5}'
total.to_csv('~/pml7/konkurs/output/' + cvs_file_name + '.csv', index=False) 
total.info(verbose=True)